In [47]:
!pip install joblib

In [1]:
data_path = 'streamlit_tutorial/housing_data_with_lat_long.parquet'

In [3]:
drop_cols = ['region','state_x','latitude','longitude','parent_metro_region','state_y','price_drops','price_drops_yoy','pending_sales_yoy','off_market_in_two_weeks_yoy']

In [5]:
import pandas as pd
df = pd.read_parquet(data_path)
drop_cols = ['region','state_x','latitude','longitude','parent_metro_region','state_y','price_drops','price_drops_yoy','pending_sales_yoy','off_market_in_two_weeks_yoy']
df = df.drop(columns=drop_cols)
df = df.dropna()
df['period_begin'] = pd.to_datetime(df['period_begin'])
df['period_end'] = pd.to_datetime(df['period_end'])
int_columns = ['median_sale_price', 'homes_sold', 'inventory', 'irs_estimated_population']
df[int_columns] = df[int_columns].astype(float)

In [8]:
from datetime import datetime 
today_date = datetime.now()
today_year = today_date.year
today_month = today_date.month


In [9]:
data = data.sort_values(by='period_end', ascending=False)

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
# Assuming your dataframe is named 'df'

# Select relevant features and target variable
features = ['zip','county','primary_city','period_year','period_month','property_type', 'median_list_price', 'median_sale_price_yoy', 'median_list_price_yoy',
            'median_ppsf', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_yoy',
            'homes_sold', 'homes_sold_yoy', 'pending_sales', 'new_listings', 'new_listings_yoy',
            'inventory', 'inventory_yoy', 'median_dom', 'median_dom_yoy', 'avg_sale_to_list',
            'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_yoy',
            'off_market_in_two_weeks', 'irs_estimated_population']

numeric_features = ['median_list_price', 'median_sale_price_yoy', 'median_list_price_yoy',
            'median_ppsf', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_yoy',
            'homes_sold', 'homes_sold_yoy', 'pending_sales', 'new_listings', 'new_listings_yoy',
            'inventory', 'inventory_yoy', 'median_dom', 'median_dom_yoy', 'avg_sale_to_list',
            'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_yoy',
            'off_market_in_two_weeks', 'irs_estimated_population']
target = 'median_sale_price'

# Selecting features and target variable
df['period_month'] = df['period_begin'].dt.month
df['period_year'] = df['period_begin'].dt.year

string_cols = ['zip','county','primary_city','property_type']
label_encoder = LabelEncoder()
for col in string_cols:
# Apply LabelEncoder to the 'property_type' column
    df[col] = label_encoder.fit_transform(df[col])

train_df = df[df['period_begin'].dt.year!=2023]
test_df = df[df['period_begin'].dt.year==2023]

# Split the data into training and testing sets
X_train,y_train, = train_df[features],train_df['median_sale_price']
X_test,y_test = test_df[features],test_df['median_sale_price']

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()


X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

model = GradientBoostingRegressor(random_state=42)

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150,200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}
param_grid = {
    'n_estimators': [50],
    'learning_rate': [0.2],
    'max_depth': [3]}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Perform grid search on the training data
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_scaled)
import joblib
joblib.dump(best_model, 'streamlit_tutorial/prediction_model_tuned.joblib')
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 2636661670.101608


In [54]:
import joblib
joblib.dump(best_model, 'streamlit_tutorial/prediction_model_tuned.joblib')

In [55]:
joblib.dump(best_model, 'prediction_model_tuned.joblib')

['prediction_model.joblib']

In [ ]:
md = joblib.load('streamlit_tutorial/prediction_model_tuned.joblib')
prd=md.predict(X_test_scaled)
mse = mean_squared_error(y_test, prd)
print(f'Mean Squared Error: {mse}')

In [57]:
train_df.shape

(94187, 30)

In [56]:
test_df.shape

(6130, 30)

In [59]:
from sklearn.metrics import mean_squared_error, r2_score

In [61]:
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, predictions))

Coefficient of determination: 0.96


In [62]:
y_test

7         347500.0
10        341666.5
18        290000.0
41        340000.0
54        325000.0
            ...   
137099    399945.0
137133    398948.0
137157    415000.0
137196    399990.0
137221    430000.0
Name: median_sale_price, Length: 6130, dtype: float64

In [63]:
predictions

array([360419.58063567, 337249.06667894, 237380.43773779, ...,
       432754.4289219 , 393573.0124122 , 438279.84431434])